In [1]:
%cd mmdetection
import mmcv

/data/Detection_proj/mmdetection


In [2]:
from mmdet.apis import set_random_seed
from mmcv import Config
from pprint import pprint
#cfg = Config.fromfile('./configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py')
#faster_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.378_20200504_180032-c5925ee5.pth
cfg = Config.fromfile('./configs/fcos/fcos_r50_caffe_fpn_gn-head_mstrain_640-800_2x_coco.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[102.9801, 115.9465, 122.7717], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 640), (1333, 800)],
        multiscale_mode='value',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[102.9801, 115.9465, 122.7717],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
 

In [3]:
import os
import os.path as osp
CLASS = ['china','us','uk','russia','japan','france','german','italy','australia','korea','other']
                
# Modify dataset type and path
cfg.data_root = '/data/flag/Images'
cfg.classes = CLASS

cfg.data.test.img_prefix = '/data/flag/Images'
cfg.data.test.ann_file = '/data/flag/flag_coco_test.json'
cfg.data.test.classes = CLASS

cfg.data.train.ann_file = '/data/flag/flag_coco_train.json'
cfg.data.train.img_prefix = '/data/flag/Images'
cfg.data.train.classes = CLASS

cfg.data.val.img_prefix = '/data/flag/Images'
cfg.data.val.ann_file = '/data/flag/flag_coco_val.json'
cfg.data.val.classes = CLASS

# modify num classes of the model in box head
cfg.model.bbox_head.num_classes = 11
#cfg.model.roi_head.mask_head.num_classes = 1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
#cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_\
#coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
cfg.load_from = 'checkpoints/fcos_r50_caffe_fpn_gn-head_mstrain_640-800_2x_coco-d92ceeea.pth'
# Set up working dir to save files and logs.
cfg.work_dir = './flag_proj/fcos'

if not os.path.exists(cfg.work_dir):
    os.makedirs(cfg.work_dir)
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.01 / 8
#cfg.lr_config.warmup = None
#log interval by step not epoch
cfg.log_config.interval = 100

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = ['bbox']
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.workflow = [('train',1),('val',1)]

#cfg.runner.max_epochs = 1

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')
cfg.dump(osp.join(cfg.work_dir, 'flag_config.py'))

Config:
dataset_type = 'CocoDataset'
data_root = '/data/flag/Images'
img_norm_cfg = dict(
    mean=[102.9801, 115.9465, 122.7717], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 640), (1333, 800)],
        multiscale_mode='value',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[102.9801, 115.9465, 122.7717],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            

In [4]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os.path as osp
import copy

# Build dataset
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-07-06 17:52:52,259 - mmdet - INFO - load checkpoint from checkpoints/fcos_r50_caffe_fpn_gn-head_mstrain_640-800_2x_coco-d92ceeea.pth
2021-07-06 17:52:52,260 - mmdet - INFO - Use load_from_local loader
2021-07-06 17:52:52,420 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.conv_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([11, 256, 3, 3]).
size mismatch for bbox_head.conv_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([11]).
2021-07-06 17:52:52,429 - mmdet - INFO - Start running, host: root@VM-116-54-centos, work_dir: /data/Detection_proj/mmdetection/flag_proj/fcos
2021-07-06 17:52:52,429 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2021-07-06 17:54:21,304 - mmdet - INFO - Epoch [1][100/1264]	lr: 4.167e-04, eta: 7:27:47, time: 0.889, data_time: 0.024, memory: 2946, loss_cls: 29.1947, loss_bbox: 0.4345, loss_centerness: 0.6085, loss: 30.2377, grad_norm: 222.8541
2021-07-06 17:55:48,781 - mmdet - INFO - Epoch [1][200/1264]	lr: 4.167e-04, eta: 7:22:50, time: 0.875, data_time: 0.003, memory: 2986, loss_cls: 0.5114, loss_bbox: 0.3287, loss_centerness: 0.5829, loss: 1.4229, grad_norm: 11.9111
2021-07-06 17:57:16,395 - mmdet - INFO - Epoch [1][300/1264]	lr: 4.167e-04, eta: 7:20:26, time: 0.876, data_time: 0.003, memory: 2986, loss_cls: 0.4566, loss_bbox: 0.2918, loss_centerness: 0.5767, loss: 1.3251, grad_norm: 10.3261
